In [ ]:
#!/usr/bin/env python3
"""
Pipeline Escalável de Machine Learning para Datasets Grandes
Suporta processamento de datasets com e sem posts de qualquer tamanho
"""

import pandas as pd
import numpy as np
import warnings
import gc
import joblib
from datetime import datetime
from pathlib import Path
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

warnings.filterwarnings('ignore')
np.random.seed(42)

# ============================================================================
# CONFIGURAÇÕES GLOBAIS
# ============================================================================

class Config:
    """Configurações do pipeline"""
    
    # Caminhos dos arquivos
    PATH_POSTS = '7_dados_unidos.csv'  # Atualizar com seu arquivo
    PATH_EMPRESAS = '6_empresas_rs_porte_sobreviveu_pandemia_enchente.csv'  # Atualizar com seu arquivo
    
    # Configurações de processamento
    CHUNK_SIZE = 10000  # Processar em chunks para economizar memória
    USE_DASK = True  # Usar Dask para datasets muito grandes (> 1GB)
    
    # Estratégia de modelagem
    STRATEGY = 'separate'  # 'unified', 'separate', 'hybrid'
    
    # Features
    DROP_FEATURES = ['cnpj_basico_str', 'cnpj', 'profile_picture_url']
    TARGET_COLS = ['sobreviveu_pandemia', 'sobreviveu_enchente']
    
    # Modelo
    TEST_SIZE = 0.2
    CV_FOLDS = 5
    RANDOM_STATE = 42
    
    # Output
    OUTPUT_DIR = Path('./outputs')
    MODEL_DIR = OUTPUT_DIR / 'models'
    DATA_DIR = OUTPUT_DIR / 'processed_data'

# ============================================================================
# CLASSE PRINCIPAL DO PIPELINE
# ============================================================================

class MLPipelineEscalavel:
    """Pipeline escalável para processar datasets grandes"""
    
    def __init__(self, config=Config()):
        self.config = config
        self.config.MODEL_DIR.mkdir(exist_ok=True, parents=True)
        self.config.DATA_DIR.mkdir(exist_ok=True, parents=True)
        
        self.dataset_posts = None
        self.dataset_empresas = None
        self.dataset_final = None
        self.models = {}
        
    # ------------------------------------------------------------------------
    # ETAPA 1: CARREGAMENTO DE DADOS
    # ------------------------------------------------------------------------
    
    def carregar_dados(self, path_posts=None, path_empresas=None, sample_size=None):
        """
        Carrega os datasets de forma eficiente
        
        Args:
            path_posts: Caminho para arquivo com dados de posts
            path_empresas: Caminho para arquivo com dados de empresas
            sample_size: Se especificado, carrega apenas uma amostra (útil para testes)
        """
        logger.info("=" * 80)
        logger.info("ETAPA 1: CARREGAMENTO DE DADOS")
        logger.info("=" * 80)
        
        # Usar caminhos fornecidos ou padrão
        path_posts = path_posts or self.config.PATH_POSTS
        path_empresas = path_empresas or self.config.PATH_EMPRESAS
        
        # Carregar dados de posts
        if Path(path_posts).exists():
            logger.info(f"Carregando dados de posts: {path_posts}")
            if sample_size:
                self.dataset_posts = pd.read_csv(path_posts, nrows=sample_size, low_memory=False)
            else:
                # Carregar em chunks se arquivo muito grande
                self.dataset_posts = self._carregar_csv_grande(path_posts)
            logger.info(f"  Posts carregados: {len(self.dataset_posts):,} linhas")
        
        # Carregar dados de empresas
        if Path(path_empresas).exists():
            logger.info(f"Carregando dados de empresas: {path_empresas}")
            if sample_size:
                self.dataset_empresas = pd.read_csv(path_empresas, nrows=sample_size, low_memory=False)
            else:
                self.dataset_empresas = self._carregar_csv_grande(path_empresas)
            logger.info(f"  Empresas carregadas: {len(self.dataset_empresas):,} linhas")
        
        # Análise inicial
        self._analisar_dados_carregados()
        
        return self
    
    def _carregar_csv_grande(self, filepath, chunk_size=None):
        """Carrega CSV grande usando chunks"""
        chunk_size = chunk_size or self.config.CHUNK_SIZE
        
        # Estimar tamanho do arquivo
        file_size_mb = Path(filepath).stat().st_size / (1024 * 1024)
        logger.info(f"  Tamanho do arquivo: {file_size_mb:.2f} MB")
        
        if file_size_mb > 500:  # Se maior que 500MB, usar chunks
            logger.info(f"  Usando leitura em chunks (chunk_size={chunk_size})")
            chunks = []
            for chunk in pd.read_csv(filepath, chunksize=chunk_size, low_memory=False):
                chunks.append(chunk)
            return pd.concat(chunks, ignore_index=True)
        else:
            return pd.read_csv(filepath, low_memory=False)
    
    def _analisar_dados_carregados(self):
        """Análise inicial dos dados carregados"""
        logger.info("\nAnálise dos dados carregados:")
        
        if self.dataset_posts is not None:
            # Preparar CNPJs
            self.dataset_posts['cnpj_limpo'] = (self.dataset_posts['cnpj']
                                                .astype(str)
                                                .str.replace('[^0-9]', '', regex=True))
            self.dataset_posts['cnpj_basico'] = self.dataset_posts['cnpj_limpo'].str[:8]
            
            cnpjs_unicos_posts = self.dataset_posts['cnpj_basico'].nunique()
            logger.info(f"  CNPJs únicos com posts: {cnpjs_unicos_posts:,}")
            logger.info(f"  Média de posts por CNPJ: {len(self.dataset_posts)/cnpjs_unicos_posts:.1f}")
        
        if self.dataset_empresas is not None:
            logger.info(f"  Total de empresas: {len(self.dataset_empresas):,}")
            if 'sobreviveu_pandemia' in self.dataset_empresas.columns:
                taxa_sobrev_pandemia = self.dataset_empresas['sobreviveu_pandemia'].mean()
                logger.info(f"  Taxa sobrevivência pandemia: {taxa_sobrev_pandemia:.1%}")
            if 'sobreviveu_enchente' in self.dataset_empresas.columns:
                taxa_sobrev_enchente = self.dataset_empresas['sobreviveu_enchente'].mean()
                logger.info(f"  Taxa sobrevivência enchente: {taxa_sobrev_enchente:.1%}")
    
    # ------------------------------------------------------------------------
    # ETAPA 2: FEATURE ENGINEERING
    # ------------------------------------------------------------------------
    
    def gerar_features(self):
        """Gera features dos datasets"""
        logger.info("\n" + "=" * 80)
        logger.info("ETAPA 2: FEATURE ENGINEERING")
        logger.info("=" * 80)
        
        features_list = []
        
        # Features de posts
        if self.dataset_posts is not None:
            features_posts = self._gerar_features_posts()
            features_list.append(('posts', features_posts))
        
        # Features de empresas
        if self.dataset_empresas is not None:
            features_empresas = self._gerar_features_empresas()
            features_list.append(('empresas', features_empresas))
        
        return features_list
    
    def _gerar_features_posts(self):
        """Gera features agregadas dos posts"""
        logger.info("\nGerando features de posts...")
        
        # Converter colunas numéricas para o tipo correto
        numeric_cols = ['followers_count', 'media_count', 'like_count']
        for col in numeric_cols:
            if col in self.dataset_posts.columns:
                self.dataset_posts[col] = pd.to_numeric(self.dataset_posts[col], errors='coerce')
        
        # Criar features derivadas se as colunas base existirem
        if 'caption' in self.dataset_posts.columns:
            self.dataset_posts['caption_length'] = self.dataset_posts['caption'].fillna('').astype(str).apply(len)
            self.dataset_posts['caption_words'] = self.dataset_posts['caption'].fillna('').astype(str).apply(lambda x: len(x.split()))
        
        if 'followers_count' in self.dataset_posts.columns and 'like_count' in self.dataset_posts.columns:
            # Calcular engagement_rate se não existir
            if 'engagement_rate' not in self.dataset_posts.columns:
                # Garantir que as colunas são numéricas
                followers = pd.to_numeric(self.dataset_posts['followers_count'], errors='coerce')
                likes = pd.to_numeric(self.dataset_posts['like_count'], errors='coerce')
                self.dataset_posts['engagement_rate'] = (
                    (likes / (followers + 1)) * 100
                ).replace([np.inf, -np.inf], np.nan)
        
        # Definir agregações apenas para colunas que existem
        agg_dict = {}
        
        # Colunas obrigatórias (verificar se existem)
        required_cols = {
            'followers_count': ['mean', 'max', 'min', 'std'],
            'media_count': ['mean', 'max', 'min'],
            'like_count': ['sum', 'mean', 'median', 'std', 'max'],
            'engagement_rate': ['mean', 'median', 'std', 'max'],
            'caption_length': ['mean', 'median', 'std', 'max'],
            'caption_words': ['mean', 'median', 'std', 'max'],
        }
        
        for col, funcs in required_cols.items():
            if col in self.dataset_posts.columns:
                agg_dict[col] = funcs
        
        # Verificar qual coluna usar para agrupamento e contagem
        groupby_col = None
        count_col = None
        if 'cnpj_basico' in self.dataset_posts.columns:
            groupby_col = 'cnpj_basico'
            count_col = 'cnpj_basico'
        elif 'cnpj' in self.dataset_posts.columns:
            groupby_col = 'cnpj'
            count_col = 'cnpj'
        else:
            logger.warning("  Nenhuma coluna CNPJ encontrada para agrupamento")
            return None
        
        # Adicionar contagem de posts
        agg_dict[count_col] = 'count'
        
        # Adicionar colunas opcionais se existirem
        optional_cols = ['likes_per_media', 'biography_length', 'biography_words', 
                        'has_emoji_bio', 'year', 'month', 'day_of_week', 'hour']
        
        for col in optional_cols:
            if col in self.dataset_posts.columns:
                if col in ['year', 'month']:
                    agg_dict[col] = ['min', 'max', 'nunique']
                elif col in ['day_of_week', 'hour']:
                    agg_dict[col] = ['mean', 'std']
                elif col == 'has_emoji_bio':
                    agg_dict[col] = ['mean']
                else:
                    agg_dict[col] = ['mean', 'std']
        
        # Verificar se há colunas para agregar
        if len(agg_dict) == 0:
            logger.warning("  Nenhuma feature pôde ser gerada")
            return None
        
        # Agregar por CNPJ
        features = self.dataset_posts.groupby(groupby_col).agg(agg_dict)
        
        # Renomear colunas
        features.columns = ['_'.join(col).replace('cnpj_count', 'total_posts').replace('cnpj_basico_count', 'total_posts') 
                           for col in features.columns]
        
        # Features derivadas
        if 'total_posts' in features.columns and 'media_count_mean' in features.columns:
            features['posts_per_media'] = features['total_posts'] / (features['media_count_mean'] + 1)
        
        if 'engagement_rate_std' in features.columns and 'engagement_rate_mean' in features.columns:
            features['engagement_consistency'] = (features['engagement_rate_std'] / 
                                                 (features['engagement_rate_mean'] + 0.001))
        
        # Calcular tendências se possível
        if 'year_min' in features.columns and 'year_max' in features.columns:
            features['temporal_span_years'] = features['year_max'] - features['year_min']
        
        logger.info(f"  Features de posts geradas: {features.shape[1]} features para {features.shape[0]} CNPJs")
        
        # Resetar index para merge
        features = features.reset_index()
        if groupby_col == 'cnpj_basico':
            features.rename(columns={'cnpj_basico': 'cnpj_basico_str'}, inplace=True)
        elif groupby_col == 'cnpj':
            features.rename(columns={'cnpj': 'cnpj_basico_str'}, inplace=True)
        
        return features
    
    def _gerar_features_empresas(self):
        """Gera features das empresas"""
        logger.info("\nGerando features de empresas...")
        
        df = self.dataset_empresas.copy()
        features = pd.DataFrame()
        
        # Identificador
        if 'cnpj_basico' in df.columns:
            features['cnpj_basico_str'] = df['cnpj_basico'].astype(str)
        
        # Features básicas
        for col in ['porte', 'situacao_cadastral', 'motivo_situacao_cadastral']:
            if col in df.columns:
                features[col] = df[col]
        
        # CNAE
        if 'cnae_fiscal_principal' in df.columns:
            features['cnae_2_digitos'] = df['cnae_fiscal_principal'].astype(str).str[:2]
            features['cnae_3_digitos'] = df['cnae_fiscal_principal'].astype(str).str[:3]
        
        # Features temporais
        if 'data_inicio_atividade' in df.columns:
            df['data_inicio_atividade'] = pd.to_datetime(df['data_inicio_atividade'], errors='coerce')
            features['idade_empresa_anos'] = (datetime.now() - df['data_inicio_atividade']).dt.days / 365.25
        
        if 'data_situacao_cadastral' in df.columns:
            df['data_situacao_cadastral'] = pd.to_datetime(
                df['data_situacao_cadastral'].astype(str), 
                format='%Y%m%d', 
                errors='coerce'
            )
            features['tempo_situacao_anos'] = (datetime.now() - df['data_situacao_cadastral']).dt.days / 365.25
        
        # Features geográficas
        if 'municipio' in df.columns:
            features['municipio_codigo'] = df['municipio']
        
        if 'cep' in df.columns:
            features['cep_3_digitos'] = df['cep'].astype(str).str[:3]
        
        # Indicadores
        if 'situacao_cadastral' in df.columns:
            features['empresa_ativa'] = (df['situacao_cadastral'] == 2).astype(int)
            features['empresa_baixada'] = (df['situacao_cadastral'] == 8).astype(int)
            features['empresa_suspensa'] = (df['situacao_cadastral'] == 4).astype(int)
        
        # Targets
        for target in self.config.TARGET_COLS:
            if target in df.columns:
                features[target] = df[target]
        
        logger.info(f"  Features de empresas geradas: {features.shape[1]} features para {features.shape[0]} empresas")
        
        return features
    
    # ------------------------------------------------------------------------
    # ETAPA 3: COMBINAÇÃO DE DATASETS
    # ------------------------------------------------------------------------
    
    def combinar_datasets(self, features_posts=None, features_empresas=None, strategy=None):
        """
        Combina datasets usando diferentes estratégias
        
        Estratégias:
        - 'unified': Um único dataset com imputação
        - 'separate': Mantém datasets separados
        - 'hybrid': Cria múltiplas versões para ensemble
        """
        logger.info("\n" + "=" * 80)
        logger.info("ETAPA 3: COMBINAÇÃO DE DATASETS")
        logger.info("=" * 80)
        
        strategy = strategy or self.config.STRATEGY
        logger.info(f"Estratégia selecionada: {strategy.upper()}")
        
        if strategy == 'unified':
            return self._combinar_unificado(features_posts, features_empresas)
        elif strategy == 'separate':
            return self._combinar_separado(features_posts, features_empresas)
        elif strategy == 'hybrid':
            return self._combinar_hibrido(features_posts, features_empresas)
        else:
            raise ValueError(f"Estratégia desconhecida: {strategy}")
    
    def _combinar_unificado(self, features_posts, features_empresas):
        """Cria dataset unificado com imputação"""
        logger.info("\nCriando dataset unificado...")
        
        # Fazer merge
        if features_posts is not None and features_empresas is not None:
            # Identificar empresas com posts
            cnpjs_com_posts = set(features_posts['cnpj_basico_str'])
            features_empresas['tem_posts'] = features_empresas['cnpj_basico_str'].isin(cnpjs_com_posts).astype(int)
            
            # Left join
            dataset = features_empresas.merge(
                features_posts,
                on='cnpj_basico_str',
                how='left',
                suffixes=('', '_posts')
            )
            
            logger.info(f"  Dataset unificado: {dataset.shape[0]} empresas, {dataset.shape[1]} features")
            logger.info(f"  Empresas com posts: {dataset['tem_posts'].sum():,}")
            logger.info(f"  Empresas sem posts: {(~dataset['tem_posts'].astype(bool)).sum():,}")
            
            # Salvar
            output_path = self.config.DATA_DIR / 'dataset_unificado.csv'
            dataset.to_csv(output_path, index=False)
            logger.info(f"  Salvo em: {output_path}")
            
            return dataset
        
        return features_empresas or features_posts
    
    def _combinar_separado(self, features_posts, features_empresas):
        """Mantém datasets separados"""
        logger.info("\nMantendo datasets separados...")
        
        datasets = {}
        
        if features_posts is not None and features_empresas is not None:
            # Dataset com posts
            cnpjs_com_posts = set(features_posts['cnpj_basico_str'])
            empresas_com_posts = features_empresas[
                features_empresas['cnpj_basico_str'].isin(cnpjs_com_posts)
            ]
            
            dataset_com_posts = empresas_com_posts.merge(
                features_posts,
                on='cnpj_basico_str',
                how='inner'
            )
            
            # Dataset sem posts
            dataset_sem_posts = features_empresas[
                ~features_empresas['cnpj_basico_str'].isin(cnpjs_com_posts)
            ]
            
            datasets['com_posts'] = dataset_com_posts
            datasets['sem_posts'] = dataset_sem_posts
            
            logger.info(f"  Dataset COM posts: {dataset_com_posts.shape}")
            logger.info(f"  Dataset SEM posts: {dataset_sem_posts.shape}")
            
            # Salvar
            for nome, df in datasets.items():
                output_path = self.config.DATA_DIR / f'dataset_{nome}.csv'
                df.to_csv(output_path, index=False)
                logger.info(f"  Salvo: {output_path}")
        
        return datasets
    
    def _combinar_hibrido(self, features_posts, features_empresas):
        """Cria múltiplas versões para ensemble"""
        logger.info("\nCriando datasets híbridos para ensemble...")
        
        datasets = {}
        
        # 1. Dataset unificado
        datasets['unificado'] = self._combinar_unificado(features_posts, features_empresas)
        
        # 2. Datasets separados
        separados = self._combinar_separado(features_posts, features_empresas)
        if isinstance(separados, dict):
            datasets.update(separados)
        
        # 3. Dataset apenas com features comuns
        if features_empresas is not None:
            datasets['base'] = features_empresas.copy()
            logger.info(f"  Dataset BASE (só empresas): {datasets['base'].shape}")
        
        return datasets
    
    # ------------------------------------------------------------------------
    # ETAPA 4: TREINAMENTO DE MODELOS
    # ------------------------------------------------------------------------
    
    def treinar_modelos(self, datasets, target='sobreviveu_pandemia'):
        """
        Treina modelos de acordo com a estratégia
        """
        logger.info("\n" + "=" * 80)
        logger.info("ETAPA 4: TREINAMENTO DE MODELOS")
        logger.info("=" * 80)
        logger.info(f"Target: {target}")
        
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import StandardScaler
        from sklearn.impute import SimpleImputer
        from sklearn.ensemble import RandomForestClassifier
        import xgboost as xgb
        import lightgbm as lgb
        from sklearn.metrics import roc_auc_score, classification_report
        
        modelos_treinados = {}
        
        # Se datasets é um DataFrame único
        if isinstance(datasets, pd.DataFrame):
            datasets = {'unico': datasets}
        
        # Treinar modelo para cada dataset
        for nome_dataset, df in datasets.items():
            if df is None or len(df) == 0:
                continue
                
            logger.info(f"\n--- Treinando modelo para dataset: {nome_dataset} ---")
            
            # Preparar dados
            features_to_drop = self.config.DROP_FEATURES + self.config.TARGET_COLS
            features_to_drop = [col for col in features_to_drop if col in df.columns]
            
            X = df.drop(columns=features_to_drop, errors='ignore')
            
            if target not in df.columns:
                logger.warning(f"  Target '{target}' não encontrado no dataset {nome_dataset}")
                continue
            
            y = df[target]
            
            # Remover colunas com muitos valores faltantes (> 90%)
            missing_threshold = 0.9
            missing_ratio = X.isnull().sum() / len(X)
            cols_to_keep = missing_ratio[missing_ratio < missing_threshold].index
            X = X[cols_to_keep]
            
            logger.info(f"  Features após limpeza: {X.shape[1]}")
            
            # Encoding de categóricas
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            for col in X.select_dtypes(include=['object']).columns:
                X[col] = X[col].fillna('missing')
                X[col] = le.fit_transform(X[col].astype(str))
            
            # Imputação
            imputer = SimpleImputer(strategy='median')
            X = pd.DataFrame(
                imputer.fit_transform(X),
                columns=X.columns
            )
            
            # Split
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.config.TEST_SIZE, 
                random_state=self.config.RANDOM_STATE, 
                stratify=y if len(np.unique(y)) > 1 else None
            )
            
            # Normalização
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Escolher modelo baseado no tamanho do dataset
            if len(X_train) < 100:
                logger.info("  Dataset pequeno: usando RandomForest")
                modelo = RandomForestClassifier(
                    n_estimators=50,
                    max_depth=3,
                    random_state=self.config.RANDOM_STATE
                )
            elif len(X_train) < 10000:
                logger.info("  Dataset médio: usando XGBoost")
                modelo = xgb.XGBClassifier(
                    n_estimators=100,
                    max_depth=5,
                    random_state=self.config.RANDOM_STATE,
                    use_label_encoder=False,
                    eval_metric='logloss'
                )
            else:
                logger.info("  Dataset grande: usando LightGBM")
                modelo = lgb.LGBMClassifier(
                    n_estimators=200,
                    max_depth=7,
                    learning_rate=0.05,
                    random_state=self.config.RANDOM_STATE,
                    verbose=-1
                )
            
            # Treinar
            modelo.fit(X_train_scaled, y_train)
            
            # Avaliar
            y_pred = modelo.predict(X_test_scaled)
            y_pred_proba = modelo.predict_proba(X_test_scaled)[:, 1]
            
            auc = roc_auc_score(y_test, y_pred_proba)
            logger.info(f"  AUC-ROC: {auc:.4f}")
            
            # Salvar modelo
            modelo_info = {
                'modelo': modelo,
                'scaler': scaler,
                'imputer': imputer,
                'features': X.columns.tolist(),
                'auc': auc,
                'dataset': nome_dataset,
                'target': target
            }
            
            modelos_treinados[f"{nome_dataset}_{target}"] = modelo_info
            
            # Salvar em disco
            model_path = self.config.MODEL_DIR / f"modelo_{nome_dataset}_{target}.joblib"
            joblib.dump(modelo_info, model_path)
            logger.info(f"  Modelo salvo: {model_path}")
        
        self.models = modelos_treinados
        return modelos_treinados
    
    # ------------------------------------------------------------------------
    # ETAPA 5: PIPELINE COMPLETO
    # ------------------------------------------------------------------------
    
    def executar_pipeline_completo(self, path_posts, path_empresas, strategy='hybrid'):
        """
        Executa o pipeline completo de ponta a ponta
        """
        logger.info("=" * 80)
        logger.info("EXECUTANDO PIPELINE COMPLETO")
        logger.info("=" * 80)
        logger.info(f"Início: {datetime.now()}")
        
        try:
            # 1. Carregar dados
            self.carregar_dados(path_posts, path_empresas)
            
            # 2. Gerar features
            features = self.gerar_features()
            features_dict = dict(features) if features else {}
            
            # 3. Combinar datasets
            datasets = self.combinar_datasets(
                features_posts=features_dict.get('posts'),
                features_empresas=features_dict.get('empresas'),
                strategy=strategy
            )
            
            # 4. Treinar modelos para cada target
            for target in self.config.TARGET_COLS:
                if isinstance(datasets, pd.DataFrame):
                    if target in datasets.columns:
                        self.treinar_modelos(datasets, target)
                elif isinstance(datasets, dict):
                    # Verificar se algum dataset tem o target
                    has_target = any(target in df.columns for df in datasets.values() if isinstance(df, pd.DataFrame))
                    if has_target:
                        self.treinar_modelos(datasets, target)
            
            logger.info(f"\nFim: {datetime.now()}")
            logger.info("Pipeline executado com sucesso!")
            
            # Resumo
            self._gerar_resumo()
            
        except Exception as e:
            logger.error(f"Erro no pipeline: {str(e)}")
            raise
        
        return self
    
    def _gerar_resumo(self):
        """Gera resumo da execução"""
        logger.info("\n" + "=" * 80)
        logger.info("RESUMO DA EXECUÇÃO")
        logger.info("=" * 80)
        
        if self.models:
            logger.info(f"\nModelos treinados: {len(self.models)}")
            for nome, info in self.models.items():
                logger.info(f"  - {nome}: AUC={info['auc']:.4f}")
        
        logger.info(f"\nArquivos salvos em:")
        logger.info(f"  - Modelos: {self.config.MODEL_DIR}")
        logger.info(f"  - Datasets: {self.config.DATA_DIR}")

# ============================================================================
# FUNÇÕES AUXILIARES
# ============================================================================

def exemplo_uso_basico():
    """Exemplo de uso básico do pipeline"""
    
    # Configurar caminhos dos seus arquivos
    PATH_POSTS = '/mnt/user-data/uploads/8_DADOS_UNIDOS_dataset_processado_10000.csv'
    PATH_EMPRESAS = '/mnt/user-data/uploads/6_empresas_rs_porte_sobreviveu_pandemia_enchente_100000.csv'
    
    # Criar e executar pipeline
    pipeline = MLPipelineEscalavel()
    pipeline.executar_pipeline_completo(
        path_posts=PATH_POSTS,
        path_empresas=PATH_EMPRESAS,
        strategy='hybrid'  # ou 'unified' ou 'separate'
    )
    
    return pipeline

def exemplo_uso_customizado():
    """Exemplo de uso customizado com controle fino"""
    
    # Configuração customizada
    config = Config()
    config.CHUNK_SIZE = 50000  # Chunks maiores para processamento mais rápido
    config.TEST_SIZE = 0.3  # Mais dados para teste
    
    # Criar pipeline com configuração customizada
    pipeline = MLPipelineEscalavel(config)
    
    # Executar etapas manualmente
    pipeline.carregar_dados(
        path_posts='seu_arquivo_posts.csv',
        path_empresas='seu_arquivo_empresas.csv',
        sample_size=10000  # Usar amostra para teste rápido
    )
    
    # Gerar features
    features = pipeline.gerar_features()
    
    # Combinar com estratégia específica
    datasets = pipeline.combinar_datasets(strategy='unified')
    
    # Treinar modelos
    modelos = pipeline.treinar_modelos(datasets, target='sobreviveu_pandemia')
    
    return pipeline, modelos

# ============================================================================
# EXECUÇÃO PRINCIPAL
# ============================================================================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════╗
    ║         PIPELINE ESCALÁVEL DE MACHINE LEARNING              ║
    ║     Processamento de Datasets Grandes com e sem Posts       ║
    ╚══════════════════════════════════════════════════════════════╝
    
    Este pipeline suporta 3 estratégias:
    
    1. UNIFIED: Cria um único dataset com imputação
       - Melhor para: Datasets balanceados
       - Vantagem: Simplicidade
    
    2. SEPARATE: Mantém datasets separados
       - Melhor para: Extreme imbalance (seu caso)
       - Vantagem: Modelos especializados
    
    3. HYBRID: Cria múltiplas versões
       - Melhor para: Ensemble e máxima performance
       - Vantagem: Robustez
    
    Executando com dados de exemplo...
    """)
    
    # Executar pipeline
    #pipeline = exemplo_uso_basico()
    
    print("""
    
    ╔══════════════════════════════════════════════════════════════╗
    ║                     PIPELINE CONCLUÍDO!                      ║
    ╚══════════════════════════════════════════════════════════════╝
    
    Para usar com seus próprios dados:
    
    1. Atualize os caminhos dos arquivos no código
    2. Escolha a estratégia (unified/separate/hybrid)
    3. Execute o pipeline
    
    Exemplo:
    
    pipeline = MLPipelineEscalavel()
    pipeline.executar_pipeline_completo(
        path_posts='seus_posts.csv',
        path_empresas='suas_empresas.csv',
        strategy='hybrid'
    )
    """)
    pipeline = MLPipelineEscalavel()
    pipeline.executar_pipeline_completo(
        path_posts='7_dados_unidos.csv',
        path_empresas='6_empresas_rs_porte_sobreviveu_pandemia_enchente.csv',
        strategy='hybrid'
    )

2025-11-13 15:37:00,233 - INFO - ================================================================================
2025-11-13 15:37:00,233 - INFO - EXECUTANDO PIPELINE COMPLETO
2025-11-13 15:37:00,234 - INFO - ================================================================================
2025-11-13 15:37:00,234 - INFO - Início: 2025-11-13 15:37:00.234555
2025-11-13 15:37:00,235 - INFO - ================================================================================
2025-11-13 15:37:00,236 - INFO - ETAPA 1: CARREGAMENTO DE DADOS
2025-11-13 15:37:00,236 - INFO - ================================================================================
2025-11-13 15:37:00,237 - INFO - Carregando dados de posts: 7_dados_unidos.csv
2025-11-13 15:37:00,238 - INFO -   Tamanho do arquivo: 2012.86 MB
2025-11-13 15:37:00,239 - INFO -   Usando leitura em chunks (chunk_size=10000)



    ╔══════════════════════════════════════════════════════════════╗
    ║         PIPELINE ESCALÁVEL DE MACHINE LEARNING              ║
    ║     Processamento de Datasets Grandes com e sem Posts       ║
    ╚══════════════════════════════════════════════════════════════╝
    
    Este pipeline suporta 3 estratégias:
    
    1. UNIFIED: Cria um único dataset com imputação
       - Melhor para: Datasets balanceados
       - Vantagem: Simplicidade
    
    2. SEPARATE: Mantém datasets separados
       - Melhor para: Extreme imbalance (seu caso)
       - Vantagem: Modelos especializados
    
    3. HYBRID: Cria múltiplas versões
       - Melhor para: Ensemble e máxima performance
       - Vantagem: Robustez
    
    Executando com dados de exemplo...
    

    
    ╔══════════════════════════════════════════════════════════════╗
    ║                     PIPELINE CONCLUÍDO!                      ║
    ╚══════════════════════════════════════════════════════════════╝
    
    Para usar co

2025-11-13 15:37:28,477 - INFO -   Posts carregados: 2,068,457 linhas
2025-11-13 15:37:28,478 - INFO - Carregando dados de empresas: 6_empresas_rs_porte_sobreviveu_pandemia_enchente.csv
2025-11-13 15:37:28,478 - INFO -   Tamanho do arquivo: 197.17 MB
2025-11-13 15:37:31,315 - INFO -   Empresas carregadas: 2,685,868 linhas
2025-11-13 15:37:31,315 - INFO - 
Análise dos dados carregados:
2025-11-13 15:37:33,591 - INFO -   CNPJs únicos com posts: 4,065
2025-11-13 15:37:33,591 - INFO -   Média de posts por CNPJ: 508.8
2025-11-13 15:37:33,592 - INFO -   Total de empresas: 2,685,868
2025-11-13 15:37:33,594 - INFO -   Taxa sobrevivência pandemia: 35.8%
2025-11-13 15:37:33,597 - INFO -   Taxa sobrevivência enchente: 49.6%
2025-11-13 15:37:33,597 - INFO - 
2025-11-13 15:37:33,598 - INFO - ETAPA 2: FEATURE ENGINEERING
2025-11-13 15:37:33,598 - INFO - ================================================================================
2025-11-13 15:37:33,599 - INFO - 
Gerando features de posts...
2025